<a href="https://colab.research.google.com/github/danielbauer1860/reviewadvisor/blob/main/generation/distilbert_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/docs/transformers/training
https://huggingface.co/docs/transformers/tasks/sequence_classification

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
!pip install transformers datasets sentencepiece torch evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import transformers
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
from datasets import load_dataset
import sentencepiece as spm
import evaluate
import torch
import numpy as np

In [5]:
dataset = load_dataset(
    "csv",
    data_files= "/content/drive/MyDrive/data/final_review_dataset.csv",
    delimiter=";"
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset['train'][7988]

{'text': 'This hotel occupies an historic building that was once the old high school in the town of Namekagon, located in northern Wisconsin. It has three floors, with 12 rooms in all, some of which are doubles, while others are triples. The rooms are pretty spacious and have all the amenities you need for a short-term stay. Each room is equipped with a full-sized bed, a desk, a coffee maker, a microwave, a TV, and a private bathroom with a bathtub. They also have a kitchen, a living room and a dining room, which is open to the public.',
 'label': 0}

In [7]:
dataset = dataset['train'].train_test_split(test_size=0.2)

In [8]:
id2label = {0: "FAKE", 1: "REAL"}
label2id = {"FAKE": 0, "REAL": 1}

In [9]:
model_name =  "distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenization and DataCollator

In [10]:
def tokenize_function(examples):

    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [11]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7998 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Evaluation metrics are loaded

In [13]:
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")
accuracy_metric= evaluate.load("accuracy")

In [14]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(precision_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(accuracy_metric.compute(predictions=preds, references = labels))
    return results

In [20]:
batch_size = 4
gradient_accumulation_steps=4
logging_steps = 100
new_model_name = "distilbert-base-fake-hotel-review-detector"
num_train_epochs = 2
learning_rate = 5e-5
evaluation_strategy = "epoch"
save_strategy = "epoch"
load_best_model_at_end = True

In [21]:
training_args = TrainingArguments(
    output_dir=new_model_name,
    logging_steps=logging_steps,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs,
    learning_rate=learning_rate,
    evaluation_strategy=evaluation_strategy,
    save_strategy=save_strategy,
    load_best_model_at_end=load_best_model_at_end,
    gradient_checkpointing=True
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [18]:
transformers.logging.set_verbosity_info()

In [23]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 7,998
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 1,000
  Number of trainable parameters = 66,955,010


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy
1,0.092000,0.082958,0.976422,0.977598,0.975783,0.976500
2,0.011500,0.088170,0.977985,0.977832,0.978579,0.978000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 4
Saving model checkpoint to distilbert-base-fake-hotel-review-detector/checkpoint-500
Configuration saved in distilbert-base-fake-hotel-review-detector/checkpoint-500/config.json
Model weights saved in distilbert-base-fake-hotel-review-detector/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-base-fake-hotel-review-detector/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-base-fake-hotel-review-detector/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If 

TrainOutput(global_step=1000, training_loss=0.08097731578350068, metrics={'train_runtime': 1112.157, 'train_samples_per_second': 14.383, 'train_steps_per_second': 0.899, 'total_flos': 2118948508901376.0, 'train_loss': 0.08097731578350068, 'epoch': 2.0})

In [24]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 4


{'eval_loss': 0.0829581469297409,
 'eval_f1': 0.9764220453875626,
 'eval_precision': 0.9775983284465735,
 'eval_recall': 0.9757827908714549,
 'eval_accuracy': 0.9765,
 'eval_runtime': 34.4007,
 'eval_samples_per_second': 58.138,
 'eval_steps_per_second': 14.535,
 'epoch': 2.0}

In [27]:
model.push_to_hub('dbauer1860/distilbert-base-fake-hotel-review-detector', create_pr=1)

Configuration saved in distilbert-base-fake-hotel-review-detector/config.json
Model weights saved in distilbert-base-fake-hotel-review-detector/pytorch_model.bin
Uploading the following files to dbauer1860/distilbert-base-fake-hotel-review-detector: pytorch_model.bin,config.json


pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dbauer1860/distilbert-base-fake-hotel-review-detector/commit/8dab05c2918b8591ae8623b1c15fad0603a131db', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='8dab05c2918b8591ae8623b1c15fad0603a131db', pr_url='https://huggingface.co/dbauer1860/distilbert-base-fake-hotel-review-detector/discussions/1', pr_revision='refs/pr/1', pr_num=1)